# Test of the clasification pipeline

In [4]:
import os

# Function to change to the parent directory
def change_to_parent_directory():
    # Check if the directory has already been changed
    if not os.environ.get('DIR_CHANGED'):
        try:
            current_dir = os.path.dirname(os.path.abspath(__file__))
        except NameError:
            current_dir = os.getcwd()
        parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
        os.chdir(parent_dir)
        os.environ['DIR_CHANGED'] = '1'
        print(f"Current working directory changed to: {os.getcwd()}")
    else:
        print("Directory has already been changed.")

# Call the function to change the working directory
change_to_parent_directory()

Current working directory changed to: /home/ihranicky/git/domainradar-clf


## Optional: Create testing dataset
Note: If you want do to this, set create_test_parquet to **True**

In [5]:
create_test_parquet = True

if create_test_parquet:
    import pandas as pd
    import numpy as np

    # List of input Parquet files along with their maximum rows and desired labels
    input_files = [
        {'file': 'testdata/2405_clftest_benign_filtered.parquet', 'max_rows': 4000, 'label': 'benign'},
        {'file': 'testdata/2405_clftest_phishing_filtered.parquet', 'max_rows': 480, 'label': 'phishing'},
        {'file': 'testdata/2405_clftest_malware_filtered.parquet', 'max_rows': 292, 'label': 'malware'},
        {'file': 'testdata/dga_2310.parquet', 'max_rows': 300, 'label': 'dga'},
    ]

    # Number of rows to select in total
    n_rows = 5072
    
    # Read the first file to get the initial columns and create the first dataframe
    first_file_info = input_files[0]
    combined_df = pd.read_parquet(first_file_info['file'])
    
    # Limit the number of rows if necessary for the first file
    if len(combined_df) > first_file_info['max_rows']:
        combined_df = combined_df.sample(n=first_file_info['max_rows'], random_state=1)
    
    # Overwrite the "label" column with the specified label for the first file
    combined_df['label'] = first_file_info['label']
    
    # Get the columns from the first dataframe
    all_columns = combined_df.columns.tolist()

    # Process the remaining files
    for file_info in input_files[1:]:
        df = pd.read_parquet(file_info['file'])
        
        # Limit the number of rows if necessary
        if len(df) > file_info['max_rows']:
            df = df.sample(n=file_info['max_rows'], random_state=1)
        
        # Overwrite the "label" column with the specified label
        df['label'] = file_info['label']
        
        # Ensure all columns from the first dataframe are present
        for col in all_columns:
            if col not in df.columns:
                df[col] = None
        
        # Align the dataframe to the columns of the first dataframe
        df = df[all_columns]
        
        # Append the dataframe to the combined dataframe
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    # Randomly select n_rows rows from the combined DataFrame
    selected_rows = combined_df.sample(n=n_rows, random_state=1)  # random_state for reproducibility
    
    # Save the selected rows to a new Parquet file
    selected_rows.to_parquet('testdata/validation.parquet')

## Run classification

In [6]:
# Specify the parquet file with the dataset for classification
test_dataset = 'testdata/validation.parquet'

# Number of domain names to classify with each run of the pipeline (0 = classify all)
CHUNK_SIZE = 30

In [13]:
import pandas as pd
import numpy as np
import shap
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from classifiers.pipeline import Pipeline
from classifiers.options import PipelineOptions
import matplotlib.pyplot as plt

# Initialize the classification pipeline
clf_options = PipelineOptions()
clf = Pipeline(clf_options)

# Read the input parquet file
input_df = pd.read_parquet(test_dataset)

# Function to map labels to 'benign' or 'malign'
def map_label(label):
    if label == 'benign':
        return 'benign'
    else:
        return f'malign ({label})'

# Function to convert labels to binary classes
def binary_label(label):
    return 'negative' if label == 'benign' else 'positive'

# Apply label mapping
input_df['mapped_label'] = input_df['label'].apply(map_label)
input_df['binary_label'] = input_df['label'].apply(binary_label)

# Ensure SHAP JavaScript initialization
#shap.initjs()

# Determine the number of chunks
CHUNK_SIZE = 1000  # Define CHUNK_SIZE
num_chunks = (len(input_df) + CHUNK_SIZE - 1) // CHUNK_SIZE if CHUNK_SIZE > 0 else 1

# Initialize counters for overall statistics
total_true_labels = []
total_pred_labels = []

# Format string for aligned output
header_format_str = "{:<3} | {:<50} | {:<18} | {:<10} | {:<10}"
data_format_str = "{:<3} | {:<50} | {:<18} | {:<10} | {:.6f}"

# Process the dataframe in chunks
for i in range(num_chunks):
    if CHUNK_SIZE > 0:
        start_idx = i * CHUNK_SIZE
        end_idx = start_idx + CHUNK_SIZE
        chunk_df = input_df[start_idx:end_idx]
    else:
        chunk_df = input_df

    # Perform your classification or processing on the working_df here
    print(f"===== Processing chunk {i+1}/{num_chunks} =====")

    chunk_without_label = chunk_df.drop(columns=['label', 'mapped_label', 'binary_label']) # Label should not be known to classifiers
    chunk_results = clf.classify_domains(chunk_without_label)

    # Collect predictions and true labels
    true_labels = chunk_df['binary_label'].values
    pred_labels = []
    for result in chunk_results:
        pred_label = 'negative' if result['aggregate_probability'] < 0.5 else 'positive'
        pred_labels.append(pred_label)

    # Update overall statistics
    total_true_labels.extend(true_labels)
    total_pred_labels.extend(pred_labels)

    # Display header for results
    print(header_format_str.format("Res", "Domain Name", "Actual Label", "Predicted", "Probability"))

    # Display results for each domain
    for idx, result in enumerate(chunk_results):
        actual_label = chunk_df.iloc[idx]['mapped_label']
        predicted_label = 'benign' if pred_labels[idx] == 'negative' else 'malign'
        domain_name = result['domain_name']
        aggregate_probability = result['aggregate_probability']
        status = "OK" if pred_labels[idx] == true_labels[idx] else "ER"
        print(data_format_str.format(status, domain_name[:50], actual_label, predicted_label, aggregate_probability))
        
        # Run debug_domain method for misclassified domains
        if status == "ER":
            print(f"Debugging misclassified domain: {domain_name}")
            #ndf_data = clf.pp.df_to_NDF(chunk_df, "phishing")  # Convert to NDF
            #debug_data = clf.clf_phishing_cnn.debug_domain(domain_name, ndf_data, chunk_df, n_top_features=10)
            
            # Print out the top n feature importances and values for each classifier
            #for classifier, data in debug_data.items():
            #    print(f"\nClassifier: {classifier}")
            #    print(f"Top {len(data['top_features'])} features for domain '{domain_name}':")
            #    for feature_info in data['top_features']:
            #        print(f"Feature: {feature_info['feature']}, Value: {feature_info['value']}, SHAP Value: {feature_info['shap_value']}")
                
                # Display the force plot for phishing_lgbm
                #if classifier == "phishing_cnn":
                #    base_value, shap_values, domain_row = data['force_plot_data']
                #    shap.force_plot(base_value, shap_values, domain_row)
                #    plt.show()

    # Calculate metrics for the current chunk
    accuracy = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels, pos_label='positive', average='binary')
    recall = recall_score(true_labels, pred_labels, pos_label='positive', average='binary')
    f1 = f1_score(true_labels, pred_labels, pos_label='positive', average='binary')

    tn, fp, fn, tp = confusion_matrix(true_labels, pred_labels, labels=['negative', 'positive']).ravel()
    false_positives = fp
    false_negatives = fn
    total_positives = tp + fp
    total_negatives = tn + fn

    fp_ratio = (false_positives / total_positives) if total_positives > 0 else 0
    fn_ratio = (false_negatives / total_negatives) if total_negatives > 0 else 0

    print(f"Chunk {i+1}/{num_chunks} metrics:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"False Positives: {false_positives} ({fp_ratio * 100:.2f}%)")
    print(f"False Negatives: {false_negatives} ({fn_ratio * 100:.2f}%)")
    print(f"===== Chunk {i+1}/{num_chunks} completed. =====")

# Calculate overall metrics
overall_accuracy = accuracy_score(total_true_labels, total_pred_labels)
overall_precision = precision_score(total_true_labels, total_pred_labels, pos_label='positive', average='binary')
overall_recall = recall_score(total_true_labels, total_pred_labels, pos_label='positive', average='binary')
overall_f1 = f1_score(total_true_labels, total_pred_labels, pos_label='positive', average='binary')

overall_tn, overall_fp, overall_fn, overall_tp = confusion_matrix(total_true_labels, total_pred_labels, labels=['negative', 'positive']).ravel()
overall_false_positives = overall_fp
overall_false_negatives = overall_fn
overall_total_positives = overall_tp + overall_fp
overall_total_negatives = overall_tn + overall_fn

overall_fp_ratio = (overall_false_positives / overall_total_positives) if overall_total_positives > 0 else 0
overall_fn_ratio = (overall_false_negatives / overall_total_negatives) if overall_total_negatives > 0 else 0

print("Overall metrics:")
print(f"Overall Accuracy: {overall_accuracy}")
print(f"Overall Precision: {overall_precision}")
print(f"Overall Recall: {overall_recall}")
print(f"Overall F1 Score: {overall_f1}")
print(f"Overall False Positives: {overall_false_positives} ({overall_fp_ratio * 100:.2f}%)")
print(f"Overall False Negatives: {overall_false_negatives} ({overall_fn_ratio * 100:.2f}%)")


===== Processing chunk 1/6 =====


KeyboardInterrupt: 

## Optional: Generate preliminary results for training the final aggregation classifier

In [9]:
import pandas as pd
import numpy as np
from classifiers.pipeline import Pipeline

# Initialize the classification pipeline
clf = Pipeline()

# Read the input parquet file
input_df = pd.read_parquet(test_dataset)

input_df = input_df.sample(frac=1).reset_index(drop=True)

#preliminary_results_df = clf.generate_preliminary_results(input_df, output_file="test_preliminary_results.parquet")
preliminary_results_df = clf.generate_preliminary_results(input_df, add_final=True)

In [10]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)
pd.options.display.float_format = '{:.6f}'.format
preliminary_results_df

,domain_name,dns_available,dns_nonzero,tls_available,tls_nonzero,ip_available,ip_nonzero,rdap_available,rdap_nonzero,geo_available,geo_nonzero,label,phishing_cnn_result,phishing_lgbm_result,malware_cnn_result,malware_xgboost_result,dga_binary_nn_result,phishing_sum,phishing_avg,phishing_prod,malware_sum,malware_avg,malware_prod,total_sum,total_avg,total_prod,badness_probability,phishing_xgboost_result,phishing_deepnn_result
0,www.technikapoznani.cz,0.600000,0.250000,1.000000,0.750000,1.000000,0.625000,1.000000,0.708333,1.000000,0.666667,benign,1.000000,0.133069,1.000000,0.007512,0.000000,1.133069,0.566535,0.133069,1.007512,0.503756,0.007512,2.140582,0.428116,0.000000,0.033794,0.015477,0.999921
1,www.latingrammy.com,0.600000,0.200000,1.000000,0.750000,1.000000,0.375000,1.000000,0.833333,1.000000,0.666667,benign,1.000000,0.032813,1.000000,0.002708,0.000000,1.032813,0.516406,0.032813,1.002708,0.501354,0.002708,2.035521,0.407104,0.000000,0.015846,0.049835,0.991765
2,mediaad-cdn.relevant-digital.com,0.600000,0.200000,1.000000,0.750000,1.000000,0.375000,1.000000,0.833333,1.000000,0.666667,benign,1.000000,0.081181,1.000000,0.002233,0.000001,1.081181,0.540590,0.081181,1.002233,0.501117,0.002233,2.083415,0.416683,0.000000,0.020965,0.002759,0.999999
3,wesarvnmouipwesawcz.pages.dev,0.900000,0.575000,1.000000,0.833333,1.000000,0.875000,1.000000,0.833333,1.000000,1.000000,phishing,1.000000,0.918846,1.000000,0.002233,0.000000,1.918846,0.959423,0.918846,1.002233,0.501117,0.002233,2.921080,0.584216,0.000000,0.971557,0.885538,1.000000
4,www.nihongo-omoshiroi.com,0.700000,0.400000,1.000000,0.791667,1.000000,0.625000,1.000000,0.625000,1.000000,0.666667,benign,1.000000,0.037004,1.000000,0.188901,0.000000,1.037004,0.518502,0.037004,1.188901,0.594450,0.188901,2.225905,0.445181,0.000000,0.061529,0.244279,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5067,cikd.ca,1.000000,0.725000,1.000000,0.750000,1.000000,0.625000,1.000000,0.833333,1.000000,0.722222,benign,0.000000,0.047576,0.000000,0.107275,0.218289,0.047576,0.023788,0.000000,0.107275,0.053638,0.000000,0.373140,0.074628,0.000000,0.027145,0.066251,0.999956
5068,www.biketone.cz,0.600000,0.250000,1.000000,0.750000,1.000000,0.375000,1.000000,0.625000,1.000000,0.666667,benign,1.000000,0.035985,0.000000,0.002791,0.000000,1.035985,0.517993,0.035985,0.002791,0.001395,0.000000,1.038776,0.207755,0.000000,0.052036,0.010897,0.965025
5069,handh.ru,1.000000,0.775000,1.000000,0.750000,1.000000,0.625000,0.958333,0.541667,1.000000,0.666667,benign,0.000000,0.112369,0.000000,0.347275,0.194811,0.112369,0.056185,0.000000,0.347275,0.173638,0.000000,0.654456,0.130891,0.000000,0.191177,0.187309,0.912881
5070,app-v2.credsolaris.com.br,0.600000,0.200000,1.000000,0.750000,1.000000,0.625000,1.000000,0.708333,1.000000,0.722222,phishing,1.000000,0.005972,0.000000,0.004388,0.000000,1.005972,0.502986,0.005972,0.004388,0.002194,0.000000,1.010360,0.202072,0.000000,0.039255,0.130806,0.999999


In [11]:
preliminary_results_df[["domain_name", "label", "badness_probability", "phishing_cnn_result", "phishing_deepnn_result", "phishing_lgbm_result", "phishing_xgboost_result", "malware_cnn_result", "malware_xgboost_result", "dga_binary_nn_result"]]

,domain_name,label,badness_probability,phishing_cnn_result,phishing_deepnn_result,phishing_lgbm_result,phishing_xgboost_result,malware_cnn_result,malware_xgboost_result,dga_binary_nn_result
0,www.technikapoznani.cz,benign,0.033794,1.000000,0.999921,0.133069,0.015477,1.000000,0.007512,0.000000
1,www.latingrammy.com,benign,0.015846,1.000000,0.991765,0.032813,0.049835,1.000000,0.002708,0.000000
2,mediaad-cdn.relevant-digital.com,benign,0.020965,1.000000,0.999999,0.081181,0.002759,1.000000,0.002233,0.000001
3,wesarvnmouipwesawcz.pages.dev,phishing,0.971557,1.000000,1.000000,0.918846,0.885538,1.000000,0.002233,0.000000
4,www.nihongo-omoshiroi.com,benign,0.061529,1.000000,1.000000,0.037004,0.244279,1.000000,0.188901,0.000000
...,...,...,...,...,...,...,...,...,...,...
5067,cikd.ca,benign,0.027145,0.000000,0.999956,0.047576,0.066251,0.000000,0.107275,0.218289
5068,www.biketone.cz,benign,0.052036,1.000000,0.965025,0.035985,0.010897,0.000000,0.002791,0.000000
5069,handh.ru,benign,0.191177,0.000000,0.912881,0.112369,0.187309,0.000000,0.347275,0.194811
5070,app-v2.credsolaris.com.br,phishing,0.039255,1.000000,0.999999,0.005972,0.130806,0.000000,0.004388,0.000000


In [12]:
preliminary_results_df[["domain_name", "label", "badness_probability"]]

,domain_name,label,badness_probability
0,www.technikapoznani.cz,benign,0.033794
1,www.latingrammy.com,benign,0.015846
2,mediaad-cdn.relevant-digital.com,benign,0.020965
3,wesarvnmouipwesawcz.pages.dev,phishing,0.971557
4,www.nihongo-omoshiroi.com,benign,0.061529
...,...,...,...
5067,cikd.ca,benign,0.027145
5068,www.biketone.cz,benign,0.052036
5069,handh.ru,benign,0.191177
5070,app-v2.credsolaris.com.br,phishing,0.039255
